In [13]:
%load_ext autoreload
%autoreload 2

In [91]:
import ast
import pandas as pd
import numpy as np
import pymorphy2
import json
from rank_bm25 import BM25Okapi
from tqdm import tqdm

In [7]:
tqdm.pandas()

In [2]:
df = pd.read_feather("data/goods.feather")

In [3]:
def standardization_characteristics(characteristics: str):
    if characteristics is None:
        return ""
    characteristics = ast.literal_eval(characteristics.lower())
    data = []
    for characteristic in characteristics:
        if "value" in characteristic:
            if len(characteristic["value"].split()) <= 3:
                if ("value" in characteristic) and ("unit" in characteristic):
                    data.append(f"{characteristic['value']} {characteristic['unit']}")
                elif characteristic["value"] in ["да", "нет"]:
                    if len(characteristic["name"].split()) <= 3:
                        data.append(characteristic['name'])
                else:
                    data.append(characteristic["value"])
    return ", ".join(list({i.strip() for i in data}))

In [4]:
df = df.drop(df.index[[126302, 170141]])

In [10]:
df['feat_text'] = df['Характеристики'].progress_apply(lambda s: standardization_characteristics(s))

100%|██████████| 356573/356573 [00:37<00:00, 9411.31it/s] 


In [11]:
df["text"] = df["Название СТЕ"].str.lower().str.strip() +  " [SEP] " + df["Характеристики"].str.strip()

In [30]:
df["cat_count"] = df.groupby('Категория')['Категория'].transform('count')

In [79]:
df["text25"] = df["Название СТЕ"] + " " + df["Категория"]

In [80]:
df["text25"].to_list()

['мяч футбольный MIKASA REGATEADOR5-G Мячи футбольные',
 'мяч волейбольный Gala Pro-Line 10 FIVB Мячи волейбольные ',
 'мяч волейбольный Mikasa MVA380K-OBL Мячи волейбольные ',
 'мяч волейбольный Wilson Super Soft Play Мячи волейбольные ',
 'Gutrend комплект расходных материалов для FUN 110 Pet Расходные материалы, комплектующие для прочего бытового оборудования',
 'Neolux FSM-05 набор фильтров для пылесоса Samsung Фильтр для пылесоса',
 'Jura 62536 жидкость для чистки автокаппучинатора, 1 л Расходные материалы, комплектующие для прочего бытового оборудования',
 'Karcher AD 3.200 1.629-662.0 промышленный пылесос Пылесос',
 'Vitek VT-1886(B) пылесос Пылесос',
 'Philips FC8471/01 PowerPro Compact безмешковый компактный\xa0 пылесос Пылесос',
 'Philips FC9170/02 пылесос Пылесос',
 'Midea VCB33A1 пылесос Пылесос',
 'Scarlett SC-VC80B07, Red пылесос Пылесос',
 'Samsung SC-20M257AWR пылесос Пылесос',
 'Samsung SC-885HH3P пылесос Пылесос',
 'Polaris PVC 1730СR, Red пылесос Пылесос',
 'Samsung 

#### Rank BM25

In [73]:
from string import punctuation

In [74]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [238]:
import pymorphy2
import re
import numpy as np
import json
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from typing import List


class BM25Model:
    def __init__(self, max_k=100, prefit=True):
        self.morph = pymorphy2.MorphAnalyzer()
        self.bm25 = None
        self.max_k = max_k
        if prefit:
            self.tokenized_corpus = np.load('tokenized_texts.npy', allow_pickle=True).tolist()
        else:
            self.tokenized_corpus = None

        
    def clear_text(self, text: str):
        text = re.sub('[",*+!:;<=>?@]^_`{|}~]', '', text)
        text = re.sub('\s\d+\s', '', text)
        return text
        
    def lemmatize(self, text):
        words = text.split() # разбиваем текст на слова
        res = list()
        for word in words:
            p = self.morph.parse(word)[0]
            nf = p.normal_form
            if nf not in res:
                res.append(nf)

        return list(res)
    
    def fit(self, corpus: List[str] = []):
        if self.tokenized_corpus is None:
            self.tokenized_corpus = [self.lemmatize(self.clear_text(sent)) for sent in tqdm(corpus)]
        self.bm25 = BM25Okapi(self.tokenized_corpus)
        
    def predict(self, query: str, top_k=None):
        '''Return top_k relevant indexes and their scores'''
        
        if self.bm25 is None:
            raise Exception("Fit model at first!")
            
        tokenized_query = self.lemmatize(self.clear_text(query))
        doc_scores = self.bm25.get_scores(tokenized_query)
        
        if top_k is None:
            n_positive = np.sum(np.array(doc_scores) > 0)
            top_k = np.max([n_positive, self.max_k])
            
        ind = np.argsort(doc_scores)[top_k:][::-1]
        scores = sorted(doc_scores, reverse=True)[:top_k]
        return ind, scores

In [230]:
bm = BM25Model(prefit=False)

In [231]:
bm.fit()

In [232]:
query = "клей универсальный"

In [233]:
ids, scores = bm.predict(query)

In [234]:
idx.shape

(350910,)

In [235]:
df.iloc[ids].head()

,ID СТЕ,Название СТЕ,Категория,Код КПГЗ,Характеристики,feat_text,text,cat_count,text25
54966,17110611,Универсальный клей,Клеи,01.11.03.07.99,"[{""Name"":""Вид лакокрасочного материала"",""Id"":-...","материал, водно-дисперсионный клей, клей, това...","универсальный клей [SEP] [{""Name"":""Вид лакокра...",921,универсальный клей клеи
169923,24012130,Клей универсальный,Клеи,01.11.03.07.99,"[{""Name"":""Упаковка"",""Id"":-200564180,""Value"":""т...","18 мес, материал, клей, туба, товары, желтый, ...","клей универсальный [SEP] [{""Name"":""Упаковка"",""...",921,клей универсальный клеи
35565,1355379,Универсальный акриловый клей,Клеи,01.11.03.07.99,"[{""Name"":""Вид продукции"",""Id"":322353980,""Value...","акриловый латекс, материал, клей, товары, лако...","универсальный акриловый клей [SEP] [{""Name"":""В...",921,универсальный акриловый клей клеи
6532,1207490,"Клей универсальный ""Секунда"", 30 мл",Клеи,01.11.03.07.99,"[{""Name"":""Тип"",""Id"":286016240,""Value"":""клей""},...","универсальный, 44.00000 г, клей, 30 см[3*];^мл","клей универсальный ""секунда"", 30 мл [SEP] [{""N...",921,"клей универсальный ""секунда"", 30 мл клеи"
219289,34189970,Бустилат Клей универсальный,Клеи,01.11.03.07.99,"[{""Name"":""Бренд"",""Id"":340299851,""Value"":""Лакра...","водная полимерная дисперсия, материал, лакра, ...","бустилат клей универсальный [SEP] [{""Name"":""Бр...",921,бустилат клей универсальный клеи


In [15]:
df["Название СТЕ"].str.lower().str.strip().str.split().to_list()

[['мяч', 'футбольный', 'mikasa', 'regateador5-g'],
 ['мяч', 'волейбольный', 'gala', 'pro-line', '10', 'fivb'],
 ['мяч', 'волейбольный', 'mikasa', 'mva380k-obl'],
 ['мяч', 'волейбольный', 'wilson', 'super', 'soft', 'play'],
 ['gutrend',
  'комплект',
  'расходных',
  'материалов',
  'для',
  'fun',
  '110',
  'pet'],
 ['neolux', 'fsm-05', 'набор', 'фильтров', 'для', 'пылесоса', 'samsung'],
 ['jura',
  '62536',
  'жидкость',
  'для',
  'чистки',
  'автокаппучинатора,',
  '1',
  'л'],
 ['karcher', 'ad', '3.200', '1.629-662.0', 'промышленный', 'пылесос'],
 ['vitek', 'vt-1886(b)', 'пылесос'],
 ['philips',
  'fc8471/01',
  'powerpro',
  'compact',
  'безмешковый',
  'компактный',
  'пылесос'],
 ['philips', 'fc9170/02', 'пылесос'],
 ['midea', 'vcb33a1', 'пылесос'],
 ['scarlett', 'sc-vc80b07,', 'red', 'пылесос'],
 ['samsung', 'sc-20m257awr', 'пылесос'],
 ['samsung', 'sc-885hh3p', 'пылесос'],
 ['polaris', 'pvc', '1730сr,', 'red', 'пылесос'],
 ['samsung', 'vcjg24lv', 'пылесос'],
 ['samsung', 'sc

In [38]:
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [28]:
corpus = df["Название СТЕ"].str.lower().str.strip().to_list()

In [40]:
tokenized_corpus = df["Название СТЕ"].str.lower().str.strip().progress_apply(lambda t: lemmatize(t)).to_list()

100%|██████████| 356573/356573 [04:38<00:00, 1280.01it/s]


In [243]:
morph = pymorphy2.MorphAnalyzer()
def clear_text(text: str):
    text = re.sub('[",*+!:;<=>?@]^_`{|}~]', '', text)
    text = re.sub('\s\d+\s', '', text)
    return text

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        nf = p.normal_form
        if nf not in res:
            res.append(nf)

    return res

In [239]:
df.shape

(356573, 9)

In [244]:
bm = BM25Model(prefit=False)

In [245]:
df = pd.read_feather("data/goods.feather")

In [246]:
df["text25"] = df["Название СТЕ"].str.lower().str.strip() + " " + df["Категория"].str.lower().str.strip()
tokenized_corpus  = df["text25"].progress_apply(lambda t: lemmatize(clear_text(t))).to_list()

100%|██████████| 356575/356575 [08:00<00:00, 742.36it/s] 


In [201]:
with open("tokenized_texts.json", "w") as fp:
     json.dump(tokenized_corpus, fp)

In [221]:
with open("tokenized_texts.json", "r") as fp:
    tokenized_corpus = json.load(fp)

In [247]:
np.save('tokenized_texts.npy', np.array(tokenized_corpus))    # .npy extension is added if not given
d = np.load('tokenized_texts.npy', allow_pickle=True)
np.array(tokenized_corpus) == d

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


array([ True,  True,  True, ...,  True,  True,  True])

In [248]:
len(d)

356575

In [223]:
np.array(tokenized_corpus).save()

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


array([list(['мяч', 'футбольный', 'mikasa', 'regateador5-g', 'мяч', 'футбольный']),
       list(['мяч', 'волейбольный', 'gala', 'pro-linefivb', 'мяч', 'волейбольный']),
       list(['мяч', 'волейбольный', 'mikasa', 'mva380k-obl', 'мяч', 'волейбольный']),
       ...,
       list(['обои', 'флизелиновый', 'под', 'покраска', 'nc', 'antivandal,', '(арт.4010-16),', '1,06*25', 'м', 'обои']),
       list(['мусорный', 'ведро', 'kimberly-clark', 'aquarius', 'белый', 'пластиковое,ть', 'контейнер', 'и', 'другой', 'ёмкость', 'для', 'мусор', 'пластмассовый']),
       list(['весы', 'cas', 'db-ii(е)', 'весы'])], dtype=object)

In [152]:
bm25 = BM25Okapi(tokenized_corpus)

In [153]:
query = "Солнцезащитные очки (!23)"
tokenized_query = lemmatize(clear_text(query))
tokenized_query

['солнцезащитный', 'очки', '23']

In [133]:
doc_scores = bm25.get_scores(tokenized_query)

In [148]:
np.sum(np.array(doc_scores) > 0)

356

In [60]:
df[df["cat_count"] == 1].iloc[-1]["Категория"]

'Пластина для фиксации для черепно-лицевой хирургии, рассасывающаяся'

In [135]:
merged_lemm_corpus = [" ".join(words) for words in tokenized_corpus]

In [149]:
bm25.get_top_n(tokenized_query, merged_lemm_corpus, n=356)

['очки солнцезащитный велосипедный с поляризация специальный экипировка для велоспорт',
 'солнцезащитный система жалюзи оконный',
 'солнцезащитный система жалюзи оконный',
 'плёнка солнцезащитный джэжэ полимерный строительный материал',
 'монтаж, установка, сборка солнцезащитный система прочее оборудование',
 'изготовление солнцезащитный изделие из комбинация различный материал',
 'монтаж, установка, сборка солнцезащитный система прочее оборудование',
 'монтаж, установка, сборка солнцезащитный система прочее оборудование',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'защитный очки',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'защитный очки',
 'очки защитный',
 'очки защитный',
 'очки защитный',
 'рентгенозащитный очки',
 'очки защитный',
 'изготовление солнцезащитный система изделие из комбинация различн

In [37]:
morph = pymorphy2.MorphAnalyzer()